In [1]:
import argparse
import os
import cv2
import numpy as np
from tqdm import tqdm
from preprocessing import parse_annotation
from utils import draw_boxes
from frontend import Vehicle
import json

Using TensorFlow backend.


In [2]:
def pred(config_path, weights_path, image_path):
    with open(config_path) as config_buffer:    
        config = json.load(config_buffer)

    model = Vehicle(backend          = config['model']['backend'],
                 input_size          = config['model']['input_size'], 
                 labels              = config['model']['labels'],
                 actions             = config['model']['actions'],
                 ob_weights          = config['model']['ob_weights'],
                 max_box_per_image   = config['model']['max_box_per_image'],
                 anchors             = config['model']['anchors'])

    model.load_weights(weights_path)

    if image_path[-4:] == '.mp4':
        video_out = image_path[:-4] + '_detected' + image_path[-4:]
        video_reader = cv2.VideoCapture(image_path)

        nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))

        video_writer = cv2.VideoWriter(video_out,
                               cv2.VideoWriter_fourcc(*'MPEG'), 
                               50.0, 
                               (frame_w, frame_h))

        for i in tqdm(range(nb_frames)):
            _, image = video_reader.read()
            
            boxes = model.predict(image)
            image = draw_boxes(image, boxes, config['model']['labels'])

            video_writer.write(np.uint8(image))

        video_reader.release()
        video_writer.release() 
    else:
        image = cv2.imread(image_path)
        nout = model.predict(image)[1]
        print(nout)
        return config['model']['actions'][np.argmax(nout)]

In [3]:
nout = pred('config.json', 'full_softmax.h5', 'data_test/images/26_1.jpg')

Instructions for updating:
Colocations handled automatically by placer.
(15, 15)
(15, 15)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
dir1 (Conv2D)                   (None, 256, 256, 2)  54          input_1[0][0]                    
__________________________________________________________________________________________________
dir2 (Conv2D)                   (None, 256, 256, 2)  36          dir1[0][0]                       
__________________________________________________________________________________________________
model_1 (Model)                 (None, 15, 15, 512)  722496      input_1[0][0]                    
___________________

In [4]:
nout

' forward_left'